In [ ]:
# Runs logistic regression models for LIWC features on cleaned, processed LIWC data for Trump speeches only
# Outcome is 'base' = 1 if speech was given at a rally

In [41]:
import re
from collections import Counter
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
import statsmodels
from statsmodels.formula.api import logit
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [42]:
data = pd.read_csv('../../Data/Intermediate Data/final_all_files_liwc.csv', engine='python')

In [51]:
data.head()

,title,date,text,source,candidate,campaign_related,city,state,1,Analytic,Clout,Authentic,Tone,WPS,Sixltr,Dic,function,pronoun,ppron,i,we,you,shehe,they,ipron,article,prep,auxverb,adverb,conj,negate,verb,adj,compare,interrog,number,quant,affect,posemo,negemo,anx,anger,sad,social,family,friend,female,male,cogproc,insight,cause,discrep,tentat,certain,differ,percept,see,hear,feel,bio,body,health,sexual,ingest,drives,affiliation,achieve,power,reward,risk,focuspast,focuspresent,focusfuture,relativ,motion,space,time,work,leisure,home,money,relig,death,informal,swear,netspeak,assent,nonflu,filler,AllPunc,Period,Comma,Colon,SemiC,QMark,Exclam,Dash,Quote,Apostro,Parenth,OtherP,uniqueid,event_type,base
0,Speech: Donald Trump Holds a Campaign Rally in...,2020-10-12,"Hello, everybody. Hello, Orlando. Hello, Sanfo...",https://www.kaggle.com/tanpengshi/donald-trump...,trump,1.0,Sanford,Florida,9331.0,34.37,92.35,40.80,76.32,12.91,15.23,88.04,55.30,19.32,12.97,2.81,3.90,3.03,1.43,1.80,6.36,5.93,11.35,10.62,5.00,6.53,2.38,21.63,4.64,1.91,1.44,2.28,2.11,6.38,4.54,1.81,0.23,0.72,0.13,15.66,0.12,0.27,0.14,1.67,10.23,1.81,1.32,1.46,1.82,1.94,3.01,2.04,0.64,1.10,0.26,0.92,0.21,0.54,0.03,0.06,12.55,4.84,1.61,4.00,2.75,0.66,5.38,13.27,2.57,14.85,2.80,7.31,5.35,2.40,0.32,0.19,0.72,0.20,0.27,0.62,0.10,0.03,0.13,0.33,0.02,24.72,7.17,9.41,0.00,0.06,0.81,0.00,1.13,0.00,5.69,0.04,0.41,2020-10-12Speech: Donald Trump Holds a Campaig...,rally,1.0
1,Speech: Donald Trump Holds a Campaign Rally in...,2020-09-30,"Hello, everybody. Hello, Duluth. Hello, Duluth...",https://www.kaggle.com/tanpengshi/donald-trump...,trump,1.0,Duluth,Minnesota,7367.0,31.41,91.66,35.76,68.79,11.81,14.12,88.12,55.35,20.62,13.49,3.33,3.41,3.18,1.83,1.75,7.13,5.58,11.17,11.01,5.14,5.36,2.35,21.87,4.40,1.95,1.67,2.38,2.32,6.64,4.44,2.16,0.15,0.77,0.48,15.46,0.19,0.26,0.53,1.94,9.09,1.83,1.21,1.06,1.49,1.67,2.38,2.33,0.95,1.25,0.14,0.79,0.19,0.42,0.00,0.03,11.73,4.24,1.63,3.95,2.63,0.71,5.82,13.44,2.78,14.17,2.58,6.41,5.32,2.28,0.45,0.33,0.77,0.29,0.19,1.07,0.16,0.00,0.30,0.53,0.07,25.94,7.10,9.52,0.01,0.00,1.55,0.00,1.85,0.00,5.48,0.0,0.43,2020-09-30Speech: Donald Trump Holds a Campaig...,rally,1.0
2,Speech: Donald Trump Holds a Campaign Rally in...,2020-09-26,"Oh, thank you. Well, we won Pennsylvania last ...",https://www.kaggle.com/tanpengshi/donald-trump...,trump,1.0,Middletown,Pennsylvania,10958.0,33.30,93.73,27.65,55.54,11.67,14.06,88.16,54.89,20.05,13.29,2.69,3.25,3.16,2.04,2.14,6.76,6.27,10.49,10.52,4.94,6.11,2.14,21.52,4.20,1.86,1.79,2.80,2.27,6.16,3.84,2.26,0.21,0.87,0.22,16.23,0.09,0.34,0.43,2.16,9.88,1.96,1.44,1.38,1.69,1.74,2.36,2.72,0.96,1.44,0.25,0.80,0.22,0.38,0.05,0.04,11.42,4.13,1.49,3.74,2.51,0.73,5.69,13.36,2.24,13.84,2.40,6.89,4.79,2.27,0.31,0.21,0.71,0.22,0.38,0.74,0.11,0.05,0.14,0.43,0.01,25.65,7.61,8.85,0.03,0.03,1.18,0.00,1.97,0.62,5.14,0.0,0.23,2020-09-26Speech: Donald Trump Holds a Campaig...,rally,1.0
3,Speech: Donald Trump Holds a Campaign Rally in...,2020-09-25,"Thank you very much. Hello, Newport News. I kn...",https://www.kaggle.com/tanpengshi/donald-trump...,trump,1.0,Newport News,Virginia,11416.0,27.55,90.93,36.27,62.97,13.31,13.23,89.00,57.65,20.81,13.46,3.22,3.74,2.72,1.76,2.02,7.35,6.35,10.92,11.90,5.14,6.31,2.41,23.34,4.48,2.07,1.66,2.02,1.77,5.45,3.70,1.73,0.14,0.56,0.18,15.80,0.19,0.38,0.41,1.94,10.72,2.30,1.24,1.84,1.77,1.61,3.14,2.55,0.79,1.52,0.20,0.77,0.17,0.25,0.25,0.04,11.16,4.40,1.25,3.57,2.39,0.59,6.58,13.93,2.37,13.83,2.51,6.89,4.57,1.70,0.45,0.24,0.61,0.25,0.21,0.89,0.12,0.03,0.29,0.42,0.02,25.14,6.26,9.87,0.02,0.03,1.31,0.00,2.00,0.00,5.48,0.0,0.18,2020-09-25Speech: Donald Trump Holds a Campaig...,rally,1.0
4,Speech: Donald Trump Holds a Campaign Rally in...,2020-09-24,"Hello Jacksonville, we love Jacksonville, I'm...",https://www.kaggle.com/tanpengshi/donald-trump...,trump,1.0,Jacksonville,Florida,9173.0,32.16,92.28,23.79,69.73,14.49,14.90,87.95,54.82,20.21,13.61,3.24,3.16,2.74,2.31,2.16,6.61,6.21,10.35,10.74,4.96,6.15,1.94,21.27,4.27,1.77

In [44]:
data.shape

(230, 104)

In [46]:
#cut these rows for now, something weird with the row alignment
data.drop(data.index[178:184], inplace=True)

In [50]:
data = data[data['candidate']=='trump']

In [ ]:
#first use statsmodels package since it provides fuller results descriptions

In [6]:
#testing for social processes - aggression//dehumanization
mod = logit('base ~ social', data).fit()

Optimization terminated successfully.
         Current function value: 0.444358
         Iterations 7


In [7]:
print(mod.summary())

                           Logit Regression Results                           
Dep. Variable:                   base   No. Observations:                  224
Model:                          Logit   Df Residuals:                      222
Method:                           MLE   Df Model:                            1
Date:                Tue, 07 Dec 2021   Pseudo R-squ.:                  0.3589
Time:                        15:07:48   Log-Likelihood:                -99.536
converged:                       True   LL-Null:                       -155.26
Covariance Type:            nonrobust   LLR p-value:                 4.697e-26
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -15.5727      2.109     -7.382      0.000     -19.707     -11.438
social         1.0795      0.144      7.511      0.000       0.798       1.361


In [8]:
mfx = mod.get_margeff()
print(mfx.summary())

        Logit Marginal Effects       
Dep. Variable:                   base
Method:                          dydx
At:                           overall
                dy/dx    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
social         0.1554      0.007     20.736      0.000       0.141       0.170


In [9]:
mod_anger = logit('base ~ social + we + anger', data).fit()

Optimization terminated successfully.
         Current function value: 0.440657
         Iterations 7


In [10]:
print(mod_anger.summary())

                           Logit Regression Results                           
Dep. Variable:                   base   No. Observations:                  224
Model:                          Logit   Df Residuals:                      220
Method:                           MLE   Df Model:                            3
Date:                Tue, 07 Dec 2021   Pseudo R-squ.:                  0.3643
Time:                        15:08:23   Log-Likelihood:                -98.707
converged:                       True   LL-Null:                       -155.26
Covariance Type:            nonrobust   LLR p-value:                 2.343e-24
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -15.7321      2.282     -6.895      0.000     -20.204     -11.260
social         1.0665      0.139      7.646      0.000       0.793       1.340
we            -0.0148      0.211     -0.070      0.9

In [11]:
# first person plural - testing aggression//in-group
mod_we = logit('base ~ we', data).fit()

Optimization terminated successfully.
         Current function value: 0.693121
         Iterations 3


In [12]:
print(mod_we.summary())

                           Logit Regression Results                           
Dep. Variable:                   base   No. Observations:                  224
Model:                          Logit   Df Residuals:                      222
Method:                           MLE   Df Model:                            1
Date:                Tue, 07 Dec 2021   Pseudo R-squ.:               3.767e-05
Time:                        15:08:28   Log-Likelihood:                -155.26
converged:                       True   LL-Null:                       -155.26
Covariance Type:            nonrobust   LLR p-value:                    0.9139
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0623      0.591     -0.105      0.916      -1.221       1.096
we             0.0166      0.153      0.108      0.914      -0.284       0.317


In [13]:
# first person sing - grandiosity (removed from paper)
mod_i = logit('base ~ i', data).fit()

Optimization terminated successfully.
         Current function value: 0.680433
         Iterations 4


In [15]:
print(mod_i.summary())

                           Logit Regression Results                           
Dep. Variable:                   base   No. Observations:                  224
Model:                          Logit   Df Residuals:                      222
Method:                           MLE   Df Model:                            1
Date:                Tue, 07 Dec 2021   Pseudo R-squ.:                 0.01834
Time:                        15:09:17   Log-Likelihood:                -152.42
converged:                       True   LL-Null:                       -155.26
Covariance Type:            nonrobust   LLR p-value:                   0.01700
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.9427      0.426     -2.215      0.027      -1.777      -0.108
i              0.3556      0.152      2.339      0.019       0.058       0.654


In [16]:
# analytical thinking - complexity
mod_analy = logit('base ~ Analytic', data).fit()

Optimization terminated successfully.
         Current function value: 0.652309
         Iterations 5


In [17]:
print(mod_analy.summary())

                           Logit Regression Results                           
Dep. Variable:                   base   No. Observations:                  224
Model:                          Logit   Df Residuals:                      222
Method:                           MLE   Df Model:                            1
Date:                Tue, 07 Dec 2021   Pseudo R-squ.:                 0.05892
Time:                        15:09:21   Log-Likelihood:                -146.12
converged:                       True   LL-Null:                       -155.26
Covariance Type:            nonrobust   LLR p-value:                 1.891e-05
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.3297      0.357      3.723      0.000       0.630       2.030
Analytic      -0.0348      0.009     -3.909      0.000      -0.052      -0.017


In [18]:
# analytical thinking + other complexity elements - complexity
mod_complex = logit('base ~ Analytic + Sixltr', data).fit()

#left out the words per sentence (WPS metric here because we believe there is some problem with this in YouTube CC)

Optimization terminated successfully.
         Current function value: 0.623304
         Iterations 6


In [19]:
print(mod_complex.summary())

                           Logit Regression Results                           
Dep. Variable:                   base   No. Observations:                  224
Model:                          Logit   Df Residuals:                      221
Method:                           MLE   Df Model:                            2
Date:                Tue, 07 Dec 2021   Pseudo R-squ.:                  0.1008
Time:                        15:09:31   Log-Likelihood:                -139.62
converged:                       True   LL-Null:                       -155.26
Covariance Type:            nonrobust   LLR p-value:                 1.605e-07
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      4.9900      1.199      4.160      0.000       2.639       7.341
Analytic       0.0173      0.017      1.019      0.308      -0.016       0.051
Sixltr        -0.3557      0.106     -3.347      0.0

In [54]:
# putting all the variables into a logistic regression

#create string for formula from list of variables
variables = list(data.columns)[11:len(data.columns)-1]

for i in range(len(variables)):
    
    try:
        formula = 'base ~' + variables[i]

        mod = logit(formula, data).fit()
        print(mod.summary())
    except:
        pass

Optimization terminated successfully.
         Current function value: 0.687310
         Iterations 4
                           Logit Regression Results                           
Dep. Variable:                   base   No. Observations:                  224
Model:                          Logit   Df Residuals:                      222
Method:                           MLE   Df Model:                            1
Date:                Sun, 12 Dec 2021   Pseudo R-squ.:                0.008421
Time:                        09:21:23   Log-Likelihood:                -153.96
converged:                       True   LL-Null:                       -155.26
Covariance Type:            nonrobust   LLR p-value:                    0.1059
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.7891      0.511      1.543      0.123      -0.213       1.791
Authentic     -0.0244      0.

Optimization terminated successfully.
         Current function value: 0.687740
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:                   base   No. Observations:                  224
Model:                          Logit   Df Residuals:                      222
Method:                           MLE   Df Model:                            1
Date:                Sun, 12 Dec 2021   Pseudo R-squ.:                0.007801
Time:                        09:21:23   Log-Likelihood:                -154.05
converged:                       True   LL-Null:                       -155.26
Covariance Type:            nonrobust   LLR p-value:                    0.1196
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.3467      0.272     -1.277      0.202      -0.879       0.185
sad            1.2025      0.

Optimization terminated successfully.
         Current function value: 0.690252
         Iterations 4
                           Logit Regression Results                           
Dep. Variable:                   base   No. Observations:                  224
Model:                          Logit   Df Residuals:                      222
Method:                           MLE   Df Model:                            1
Date:                Sun, 12 Dec 2021   Pseudo R-squ.:                0.004178
Time:                        09:21:23   Log-Likelihood:                -154.62
converged:                       True   LL-Null:                       -155.26
Covariance Type:            nonrobust   LLR p-value:                    0.2547
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.2068      1.076      1.122      0.262      -0.901       3.315
space         -0.1768      0.

/Users/heidiwallace/opt/anaconda3/envs/anlp/lib/python3.8/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/heidiwallace/opt/anaconda3/envs/anlp/lib/python3.8/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Using sklearn's model for prediction/classification

In [21]:
#next using sklearn to incorporate all the features
from sklearn.model_selection import train_test_split
from sklearn import linear_model

In [22]:
X = data.iloc[:, 9:-3]
Y = data.iloc[:, -1]

In [23]:
trainX, devX, trainY, devY = train_test_split(X, Y, test_size=0.2, random_state=0)

In [24]:
clf = linear_model.LogisticRegression(C=100, solver='lbfgs', penalty='l2', max_iter=10000)
clf.fit(trainX, trainY)

LogisticRegression(C=100, max_iter=10000)

In [25]:
print("Accuracy: %.3f" % clf.score(devX, devY))

Accuracy: 0.933


In [32]:
clfCV = linear_model.LogisticRegressionCV(max_iter=10000)
clfCV.fit(trainX, trainY)

LogisticRegressionCV(max_iter=10000)

In [33]:
print("Accuracy: %.3f" % clfCV.score(devX, devY))

Accuracy: 0.933


In [26]:
def majority_class(trainY, devY):
    labelCounts=Counter()
    for label in trainY:
        labelCounts[label]+=1
    majority_class=labelCounts.most_common(1)[0][0]
    
    correct=0.
    for label in devY:
        if label == majority_class:
            correct+=1
            
    print("%s\t%.3f" % (majority_class, correct/len(devY)))

In [27]:
majority_class(trainY, devY)

1.0	0.444


In [98]:
cdf = pd.DataFrame(clf.coef_[0], list(trainX.columns), columns=['Coefficients'])
print(cdf)

              Coefficients
Analytic          0.200778
Clout             0.516263
Authentic         0.259825
Tone              0.289461
WPS               0.001184
Sixltr            0.133143
Dic              -1.765187
function          0.049423
pronoun           0.089305
ppron             1.282238
i                 0.422348
we                0.490430
you               1.931134
shehe            -0.429785
they             -1.128961
ipron            -1.133438
article           3.770907
prep             -1.306150
auxverb          -0.417354
adverb            0.850531
conj             -1.553517
negate            1.216372
verb              1.301890
adj               1.322465
compare           1.063694
interrog         -0.149105
number            4.806803
quant            -2.330354
affect           -1.487223
posemo           -2.344707
negemo            0.764202
anx              -0.348226
anger            -1.025141
sad               0.397975
social            0.936255
family            0.161273
f

Using sklearn for CV classification on our complexity scores

In [56]:
#for complexity
X = data.loc[:, ['Analytic', 'Sixltr'] ]
Y = data.iloc[:, -1]

In [57]:
trainX, devX, trainY, devY = train_test_split(X, Y, test_size=0.2, random_state=0)

In [58]:
clfCV = linear_model.LogisticRegressionCV(max_iter=10000)
clfCV.fit(trainX, trainY)

LogisticRegressionCV(max_iter=10000)

In [59]:
print("Accuracy: %.3f" % clfCV.score(devX, devY))

Accuracy: 0.600


In [69]:
#for social
X = data.loc[:, ['social'] ]
Y = data.iloc[:, -1]

In [70]:
trainX, devX, trainY, devY = train_test_split(X, Y, test_size=0.2, random_state=0)

In [71]:
clfCV = linear_model.LogisticRegressionCV(max_iter=10000)
clfCV.fit(trainX, trainY)

LogisticRegressionCV(max_iter=10000)

In [72]:
print("Accuracy: %.3f" % clfCV.score(devX, devY))

Accuracy: 0.956


Use log odds to look at most predictive